не успеваю нормально реализовать mvp

In [121]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import StratifiedKFold

import pandas as pd
import numpy as np
from src.config import BaseConfig
from src.modeling.metrics import weighted_accuracy

config = BaseConfig()

In [122]:
path = config.processed_data_dir / "final_data_to_train.json"
test_keys_path = config.processed_data_dir / "test_tasks.csv"
df = pd.read_json(path)
test_keys = pd.read_csv(test_keys_path)

df.head(0)

,assignee_level_order,jira_key,weeks_since_member_join,time_to_complete_hours,task_text


In [123]:
test_keys.head(0)

,jira_key


## Regression type of task

In [124]:
df["time_to_complete_hours_transformed"] = df["time_to_complete_hours"].clip(2, 12)  # clip between 2 and 24 hours
df["time_to_complete_hours_transformed"] = df["time_to_complete_hours_transformed"] // 2 * 2  # round to the nearest even number

y = df["time_to_complete_hours_transformed"]
x = df["task_text"]

y = y[~df.jira_key.isin(test_keys.jira_key)]
x = x[~df.jira_key.isin(test_keys.jira_key)]

y_test_from_manual = y[~df.jira_key.isin(test_keys.jira_key)]
x_test_from_manual = x[~df.jira_key.isin(test_keys.jira_key)]

y.value_counts().sort_index()

time_to_complete_hours_transformed
2     179
4      98
6      23
8      77
10      3
12     73
Name: count, dtype: int64

In [125]:
pipe = Pipeline([
    ("tfidf", CountVectorizer(ngram_range=(2, 2))),  # ngram_range=(1, 2) means that we will use unigrams and big
    ("lr", LinearRegression()),
])

metrics = {
    "w_acc_test_transformed": [],
    "w_acc_train_transformed": [],
    "w_acc_test": [],
    "w_acc_train": []
}
skf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for train_index, test_index in skf.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pipe.fit(x_train, y_train)

    # evaluation
    keys = ["w_acc_test_transformed", "w_acc_train_transformed", "w_acc_test", "w_acc_train"]
    xs = [x_test, x_train, x_test, x_train]
    ys = [y_test, y_train, df.loc[test_index, "time_to_complete_hours"], df.loc[train_index, "time_to_complete_hours"]]
    for key, x_, y_ in zip(keys, xs, ys):
        y_pred = pipe.predict(x_)
        metrics[key].append(weighted_accuracy(y_, y_pred))

metrics = {
    k: {
        "mean": np.mean(v),
        "std": np.std(v)
    }
    for k, v in metrics.items()
}

pd.DataFrame(metrics)

,w_acc_test_transformed,w_acc_train_transformed,w_acc_test,w_acc_train
mean,0.136507,0.999375,-0.158681,-0.310316
std,0.017799,0.000879,0.066955,0.036505


In [126]:
preds = pd.DataFrame(
    {
        "y_true": y_test_from_manual,
        "y_pred": pipe.predict(x_test_from_manual)
    }
)
preds.groupby("y_true")["y_pred"].mean()

y_true
2     2.996040
4     4.472802
6     5.978106
8     7.167060
10    8.849861
12    9.883551
Name: y_pred, dtype: float64

In [127]:
preds.corr()

,y_true,y_pred
y_true,1.000000,0.822681
y_pred,0.822681,1.000000


In [128]:
weighted_accuracy(preds["y_true"], preds["y_pred"])

0.7163050534010178

### Weights evaluation

In [129]:
words = pipe.named_steps["tfidf"].get_feature_names_out()
weights = pipe.named_steps["lr"].coef_

weights_regr = pd.DataFrame({"word": words, "weight": weights, "abs_weight": abs(weights)})
weights_regr = weights_regr.sort_values("abs_weight", ascending=False)
weights_regr.head(10)

,word,weight,abs_weight
10941,issues or,-0.335485,0.335485
4636,comprehensive analysis,0.323863,0.323863
4755,conduct comprehensive,0.282622,0.282622
8931,fraud prevention,0.246522,0.246522
8922,fraud detection,0.241604,0.241604
1700,and requirements,0.233201,0.233201
8429,for all,0.232766,0.232766
965,analysis results,0.223363,0.223363
11965,metrics and,0.219873,0.219873
931,analysis of,0.218984,0.218984


In [131]:
# save model
import joblib

joblib.dump(pipe, config.models_dir / "regression_model.pkl")

['/Users/victor.barbarich/PycharmProjects/personal-projects/mlsd-course-2024/models/regression_model.pkl']

In [133]:
# test
pipe_loaded = joblib.load(config.models_dir / "regression_model.pkl")
pipe_loaded.predict(x_test_from_manual) == pipe.predict(x_test_from_manual)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,